In [3]:
# importing all the required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import STOPWORDS
import numpy as np
import string
from nltk import ngrams
from collections import defaultdict
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, precision_score, recall_score

# this is defined to clean the data and preprocess it and developed by the team
# from main_function import X_clean

# to filter all the warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
# loading the dataset into a df
df = pd.read_csv(r'C:\Users\Legion\Desktop\Disaster Detection System\train.csv')

In [5]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# checking total rows and columns of the df
df.shape

(7613, 5)

In [8]:
# checking the not null values and the data type present across all columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [12]:
# count of null values
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [10]:
# checking the unique values present for each column
df.nunique()

id          7613
keyword      221
location    3341
text        7503
target         2
dtype: int64

In [14]:
# dropping the id column since it is not required for the model
df.drop(columns='id', inplace = True)

In [15]:
# checking columns present in the df
df.columns

Index(['keyword', 'location', 'text', 'target'], dtype='object')

### Target Variable

In [16]:
# checking the target variable
df['target'].unique()

array([1, 0], dtype=int64)

In [17]:
df_target = df['target']

In [18]:
df_target.shape

(7613,)

In [19]:
df_target.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

### Exploratory Data Analysis (EDA)

In [20]:
for col in ['keyword', 'location']:
    df[col] = df[col].fillna(f'no {col}')

In [22]:
df.isna().sum()

keyword     0
location    0
text        0
target      0
dtype: int64